In [5]:
from random import randint
import numpy as np
import pandas as pd
import json
import time
import torch
import torchmetrics


In [6]:
#we have a 20 by 20 matrix, with a body and a head covering 3+ squares. We can represent the head,tail and food in a 20x20x3(channels) array
j = open('snakeLogs.json')
data = json.load(j)

In [7]:
def CreateBigArrayOfFramesFromManyGames(array_max, data):
    
    def create_body_head_food(array_max, instance,data): #instance x 20x20 (two arrays)
        plays = len(data[instance]['body'])
        image = np.zeros((plays,20,20,3))

        for key, value in data[instance]['body'].items():
            for idx, point in enumerate(value):
                y,x = point[0]-1,point[1]-1 #-1 because the stored array starts at 1, instead of 0
                if idx != 0:            #tail
                    image[int(key),y,x,0] =1
                if idx == 0:            #head
                    image[int(key),y,x,1] =1          

        for key, value in data[instance]['FoodPos'].items():    #food
            y,x = value[0]-1,value[1]-1
            image[int(key),y,x,2] =1

        return image
    
    AllGamesImage = np.zeros((array_max,20,20,3))
    frames_start=0
    for game in data.keys():
        tempArray = create_body_head_food(array_max, game,data) 
        frames_end = frames_start+ (len(tempArray)) #to create start and end for storing the x values
        if frames_end > array_max:
            return AllGamesImage, frames_start
        AllGamesImage[frames_start:frames_end] = tempArray
        frames_start = frames_end #set frames start to frames end for next iteration
    return AllGamesImage, frames_end

def CreateBigArrayofMovesFromManyGames(data):
    
    def move_made(instance,data):
        return np.array(list(data[instance]['FaceDirectionChoice'].values()))        
    
    AllGamesMoves = np.zeros((1,))
    for game in data.keys():
        tempArray = move_made(game, data)
        AllGamesMoves = np.append(AllGamesMoves, tempArray, axis=0)
    return AllGamesMoves[1:]

def CreateBigArrayofRewardFromManyGames(data):
    
    def reward(instance,data):
        ListOfReward = (list(data[instance]['CurrentScore'].values()))
        RewardArray = np.zeros((len(ListOfReward),))+max(ListOfReward)
        return RewardArray

    AllGamesReward = np.zeros((1,))
    for game in data.keys():
        tempArray = reward(game, data)
        AllGamesReward = np.append(AllGamesReward, tempArray, axis=0)
    return AllGamesReward[1:]

def CreateBigArrayOfStartingDirectionFromManyGames(data):
    
    def DirectionPointing(instance,data): 
        return np.array(list(data[instance]['StartingDirection'].values()))        
    
    AllGamesDirectionPointing=np.zeros((1,))
    for game in data.keys():
        tempArray = DirectionPointing(game,data)
        AllGamesDirectionPointing = np.append(AllGamesDirectionPointing,tempArray, axis=0)
    return AllGamesDirectionPointing[1:]

In [8]:
TimerStart = time.time()
image,frames =  CreateBigArrayOfFramesFromManyGames(1000000,data)
print(f'Time Taken: {int(time.time()-TimerStart)}')
moves =  CreateBigArrayofMovesFromManyGames(data)
reward = CreateBigArrayofRewardFromManyGames(data)
DirectionPointing = CreateBigArrayOfStartingDirectionFromManyGames(data) 
image, moves, reward, DirectionPointing =image[:frames], moves[:frames], reward[:frames], DirectionPointing[:frames]
print(f'Time Taken: {int(time.time()-TimerStart)}')

Time Taken: 0
Time Taken: 0


In [9]:
image.shape,moves.shape,reward.shape,DirectionPointing.shape

((22685, 20, 20, 3), (22685,), (22685,), (22685,))

#### Overview of Variables
----
Trainable X (State): Body, Direction Pointining

Non-trainable X (Decision): Moves

Target Y: reward

In [33]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, image, StartingDirection, move, y):
#         assert len(X) == len(y), "Data and labels have to be of equal length!"
        self.image = image.astype('int')
        self.StartingDirection = StartingDirection.astype('int')
        self.move = StartingDirection.astype('int')
        self.y = y.astype('int')

    # Not dependent on index+
    def __getitem__(self, index):
        return self.image[index],self.StartingDirection[index],self.move[index], self.y[index]

    def __len__(self):
        return len(self.y)

In [39]:
dataset = Dataset(image, moves, DirectionPointing,reward)


In [48]:
def DatasetsAndDataloaders(dataset, train_perc, val_perc, batch_size):
    
    def create_TTV_splits(train_perc, val_perc,dataset):
        assert train_perc + val_perc < 1, 'val and train percent should add up to <1'
        length = len(dataset)
        trainSize = int(train_perc * length)
        TestValSize = int(length - trainSize)
        valSize = int(val_perc * length)
        TestSize = int(TestValSize - valSize)

        train_dataset, validation_dataset = torch.utils.data.random_split(
            dataset, [trainSize, TestValSize])  ## split into 1000 training & 797 validation
        validation_dataset, test_dataset  = torch.utils.data.random_split(
            validation_dataset, [valSize, TestSize])  ## get test set from validation set

        return train_dataset, validation_dataset, test_dataset

    def create_dataloaders(train_dataset, validation_dataset, test_dataset, batch_size):
        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, )   ### dataloader batches the data
        val_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size)
        test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)
        return train_dataloader, val_dataloader, test_dataloader
    
    datasets, dataloaders = {}, {}
    train_dataset, validation_dataset, test_dataset = create_TTV_splits(train_perc, val_perc, dataset)
    train_dataloader, val_dataloader, test_dataloader = create_dataloaders(train_dataset, validation_dataset, test_dataset, batch_size)
    
    datasets['train'], datasets['val'], datasets['test'] = train_dataset, validation_dataset, test_dataset
    dataloaders['train'], dataloaders['val'], dataloaders['test'] = train_dataloader, val_dataloader, test_dataloader
    return datasets, dataloaders

In [49]:
datasets, dataloaders = DatasetsAndDataloaders(dataset, 0.7,0.2,100)

In [51]:
dataloaders['train']